# Automatic Evaluator

In [63]:
%%capture
#########################
# Requirements and Imports
#########################
!pip install -e ..

import inspect
import importlib.util
import sys

import ipywidgets as widgets
from pathlib import Path

import torch
import numpy as np
import random

import hackatari

import game_execution

In [64]:
##############################################
# Default Value Initialization
##############################################

games = [x.name.replace('.py', '') for x in Path('../hackatari/games').glob('**/*') if x.is_file() and '__init__' not in x.name and 'cpython' not in x.name]
games.sort()
games = [x.capitalize() for x in games]

game = 'pong'
model_path = 'model.gz'
log_path = 'log.json'
game_modifications = ['']
selected_modifications = ['']
active_mods = []
agents = []
rewardfunc_path = None
dopamine_pooling = False
game_mode = 0
difficulty = 0
render_mode = "None"
obs_mode = "dqn"
mode = "ram"
hud = False
renderc_oc_overlay = True
buffer_window_size = 4
frameskip = 4
repeat_action_probability = 0.25
full_action_space = False
episodes = 10


run = False # required so that game is not run on load

In [65]:
##############################################
# Helper Methods
##############################################

def set_seed(reproducibility: bool=True, seed_value: int =0):
    if reproducibility:
        random.seed(seed_value)
        torch.manual_seed(seed_value)
        np.random.seed(seed_value)
        
        # Use deterministic Algorithms
        torch.use_deterministic_algorithms(True)
        torch.utils.deterministic.fill_uninitialized_memory = True

def set_model_path(model_path_value='model.gz'):
    global agents
    model_path = Path(model_path_value)
    if model_path.is_dir():
        agents = [str(x.resolve()) for x in model_path.glob('*')  if x.is_file() and '.gz' in x.name]
    else:
        agents = [str(model_path.resolve())]
    
def set_log_path(log_path_value='log.json'):
    global log_path
    log_path = log_path_value

def set_modifications(modifications_value):
    global selected_modifications
    global active_mods
    selected_modifications = modifications_value
    active_mods = selected_modifications

def get_game_modifications_for(game: str = 'pong'):
    global game_modifications
    try:
        game_modifications = [mod.split(':')[0][1:] for mod in hackatari.core._available_modifications(game).split('*')[1:]]
    except AttributeError:
        game_modifications = []
    return game_modifications

def set_episodes(episodes_value = 3):
    global episodes
    episodes = episodes_value

def run_game():
    global agents
    global active_mods
    global rewardfunc_path
    global dopamine_pooling
    global game_mode
    global difficulty
    global render_mode
    global obs_mode
    global mode
    global hud
    global renderc_oc_overlay
    global buffer_window_size
    global frameskip
    global repeat_action_probability
    global full_action_space
    global episodes

    game_execution.eval_run(
        game = game,
        agents = agents,
        modifications = active_mods,
        #rewardfunc_path= rewardfunc_path,
        #dopamine_pooling= dopamine_pooling,
        #game_mode= game_mode,
        #difficulty= difficulty,
        #render_mode= render_mode,
        #obs_mode= obs_mode,
        #mode= mode,
        #hud= hud,
        #renderc_oc_overlay= renderc_oc_overlay,
        #buffer_window_size= buffer_window_size,
        #frameskip= frameskip,
        #repeat_action_probability= repeat_action_probability,
        #full_action_space= full_action_space,
        episodes = episodes,
    )

In [75]:
##############################################
# Widget Creation, interactivity, layout
##############################################

# Create Widgets
reproducibility_widget = widgets.Checkbox(
    value = True,
    description = "Reproducibility",
    disabled = False
)

seed_widget = widgets.IntText(
    value=0,
    placeholder=0,
    description="Seed:",
    disabled=False
)

model_path_widget = widgets.Text(
    value='model.gz',
    placeholder='Model Path',
    description="Model Path:",
    disabled=False
)

log_path_widget = widgets.Text(
    value='log.json',
    placeholder='Log Path',
    description="Log Path:",
    disabled=False
)

game_selector_widget = widgets.Select(
    options = games,
    value = "Pong",
    description = 'Game:',
    disabled = False,
)

modification_selector_widget = widgets.SelectMultiple(
    options = get_game_modifications_for(game),
    value = [],
    description = "Game Modification:",
    disabled = False
)

run_button_widget = widgets.Button(
    description = 'Run',
    disabled = False,
    buttan_style = '',
    tooltip = 'Run',
    icon = 'play'
)

episodes_widget = widgets.IntText(
    value = 3,
    placeholder = 3,
    description = 'Episodes: ',
    disabled = False
)

def update_game_selection(game_value):
    global game
    mods = get_game_modifications_for(game_value)
    game = game_value
    modification_selector_widget.options = mods

def run_log_update():
    global run
    if run:
        run_game()
    run = True


# Create Views
reproducibility_view = widgets.interactive(set_seed, reproducibility=reproducibility_widget, seed_value=seed_widget)
model_path_view = widgets.interactive(set_model_path, model_path_value=model_path_widget)
log_path_view = widgets.interactive(set_log_path, log_path_value=log_path_widget)
episodes_view = widgets.interactive(set_episodes, episodes_value=episodes_widget)
modification_selector_view = widgets.interactive(set_modifications, modifications_value=modification_selector_widget)
game_selector_view = widgets.interactive(update_game_selection, game_value=game_selector_widget)
run_view = widgets.interactive(run_log_update)

    
reproducibility_widget.observe(lambda x: reproducibility_view.update(), 'value')
seed_widget.observe(lambda x: reproducibility_view.update(), 'value')
model_path_widget.observe(lambda x: model_path_view.update(), 'value')
episodes_widget.observe(lambda x: episodes_view.update(), 'value')
log_path_widget.observe(lambda x: log_path_view.update(), 'value')
game_selector_widget.observe(lambda x: game_selector_view.update(), 'value')
run_button_widget.on_click(lambda x: run_view.update())

grid = widgets.GridspecLayout(15,1, height='900px', layout=widgets.Layout(justify_content='center'))
grid[0:2,0] = reproducibility_view
grid[2,0] = model_path_view
grid[3,0] = log_path_view
grid[4,0] = episodes_view
grid[5:7,0] = game_selector_view
grid[7:9,0] = modification_selector_view
grid[9,0] = run_button_widget
grid[10:,0] = run_view

In [76]:
##############################################
# Make the App
##############################################

app_contents = [grid]
app = widgets.VBox(app_contents, layout=widgets.Layout( max_width='2048px', margin='0 auto 0 auto'))

In [77]:
display(grid)

GridspecLayout(children=(interactive(children=(Checkbox(value=True, description='Reproducibility'), IntText(va…